# You Only Look Once: Unified, Real-Time Object Detection

paper(15.06) https://arxiv.org/abs/1506.02640<br>
code https://pjreddie.com/darknet/yolo/

---

# Abstract

Object Detection 분야의 모델은 크게 One-Stage 계열의 YOLO 모델과, Two-Stage 계열의 R-CNN 모델로 분류할 수 있다. 그 중 YOLO 모델을 탐구함으로써, One-Stage의 원리와 그에 따른 장단점을 살펴보고자 한다.

**"we frame object detection as a regression problem to spatially separated bounding boxes and associated class probabilities."**<br>
해당 시기에 Object Detection 접근법과는 달리 YOLO는 Object Detection 문제를 regression problem으로 정의하고 해결해나간다.

**"Since the whole detection pipeline is a single network, it can be optimized end-to-end directly on detection performance."**<br>
특히 YOLO는 One-Stage의 단일 pipeline으로 구성하여 end-to-end train을 가능하도록 한 것이 특징이다.

**"YOLO makes more localization errors but is less likely to predit false positives on background."**<br>
논문의 저자는 YOLO 모델이 다른 Object Detection 모델보다 localization 성능에 있어서 더 많은 error를 일으키지만, 그 효과가 미비하다고 언급한다.

# 1. Introduction

**"We reframe object detection as a single regression problem, straight from image pixels to bounding box coordinates and class probablities."**<br>
마치 사람이 한 눈에 시야를 살펴보고 물체를 인식하는 것처럼, 기존 모델들(DPM, R-CNN)과는 달리 한 번에 classification과 localization을 수행하는 YOLO의 구조를 언급한다.

YOLO의 unifed model이라는 특징은 몇몇 장점들을 가지고 있다.<br>
<br>
**"First, YOLO is extremely fast"**<br>
YOLO는 45fps의 성능을 보이며, fast version은 150fps 이상의 성능을 보인다고 언급한다.<br>
**"Second, YOLO reasons globally about the images when making predictions."**<br>
sliding window 방식이나 region proposal-based techniques과는 달리 training동안 전체 이미지를 한 번에 본다.<br>
**"Third, YOLO learns generalizable representations of objects."**<br>

# 2. Unified Detection

**"Our system divides the input image into an SxS grid. If the center of an object falls into a grid cell, that grid cell is responsible for detecting that object."**<br>
YOLO는 input image를 SxS 그리드 셀로 나눈 뒤, 각 그리드 셀이 여러 개의 Bounding Box를 가지고 물체를 예측하는 구조이다. 이 때, 어떤 물체의 중심이 특정 그리드 셀에 포함된다면, 해당 그리드 셀은 그 물체를 예측할 것을 기대할 수 있다.

**"Each grid cell predicts B bounding boes and confidence scores for those boxes. These confidence scores reflect how confidence the model is that the box contains an object and also how accurate it thinks the box is that it predicts."**<br>
각 그리드 셀은 B개의 Bounding Box를 가지고 있다는 특징에 관한 설명이다. 더 자세한 구조는 아래 내용을 통해 확인할 수 있다.

**"Each bounding box consists of 5 predictions: x, y, w, h, and confidence. The (x,y) coordinates represent the center of the box relative to the bounds of the grid cell. The width and height are predicted relative to the whole image. Finally the confidence prediction represents the IOU between the predicted box and any ground truth box.<br>
Each grid cell also predicts C conditional class probabilites, Pr(Class(i)|Object)."**<br>
다시 정리하자면, YOLO는 input image를 SxS 그리드 셀로 나눈다. 그리고 각 그리드 셀은 B개의 Bounding Box와 C개의 conditional class probability를 가지고 있다. 이는 그리드 셀에 물체가 있다고 가정했을 때, 각 class 별로 존재할 확률을 의미한다. 그리고 각 Bounding Box는 x, y, w, h, confidence를 가지고 있다. 따라서 예측에 사용되는 feature map은 S x S x ( Bx5 + C) tensor로 구성되어 있다는 것을 확인할 수 있다. 주의할 점은 각 Bounding Box별로 conditional class probability를 가지고 있는 것이 아니라, 각 그리드 셀 별로 가지고 있다는 점이다. 즉, 특정 그리드 셀 내의 모든 Bounding Box는 conditional class probability에 대한 정보를 공유하고 있다.

### 2.1. Network Design

<img src = 'https://i.imgur.com/BL31OQu.png' width='100%'>

* Convolutional neural network
* PASCAL VOC detection dataset
* 24 convolutional layers + 2 fully connected layers
* 1x1 reduction layers

### 2.2. Training